# Ipyreact Tutorial

Welcome to this ipyreact walkthrough.

# Here you will learn:
* How to use the %react cell magic
* How to write a widegt
* How to load css for your widget
* how to add parameters to your widgets
* how to interact with these parameters

The tuturial will be based on a very simple react button to show all the ipyreact features.  
First, we will use the `%react` magic from ipyreact.  
The following line registers the cellmagic:

In [1]:
%load_ext ipyreact

In [2]:
%%react

import * as React from "react";

export default function Square() {
  return (<button> X </button>);
  }

Widget()

That worked! Next, we convert this into a widget.
For that, we need the code in a _ems string inside a class that inherits from `ipyreact.ReactWidget`

In [3]:
import ipyreact

class MyExampleWidget(ipyreact.ReactWidget):
    _esm = """
    import * as React from "react";

    export default function Square() {
        return <button> X </button> 
        };"""
MyExampleWidget()

MyExampleWidget()